In [1]:
from hipecta.data import ctaTelescope2Matrix
from hipecta import core
import numpy as np
import h5py
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, ConcatDataset, sampler
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import hipecta.plots as plots
from torchvision import transforms, utils
import math
import time
import os
import sys

converter_path = os.path.abspath(os.path.join('/home/jacquemm/GammaLearn/converter_hdf5'))
if converter_path not in sys.path:
    sys.path.append(converter_path)
    
from converter_hdf5 import *
from datasets import *

In [5]:
file_1 = '/home/jacquemm/hdf5/gamma_diffuse/LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0000.hdf5'
f_1 = h5py.File(file_1, 'r')
file_2 = '/home/jacquemm/hdf5/gamma_diffuse/LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0001.hdf5'
f_2 = h5py.File(file_2, 'r')
file_3 = '/home/jacquemm/hdf5/gamma_diffuse/LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0002.hdf5'
f_3 = h5py.File(file_3, 'r')
file_4 = '/home/jacquemm/hdf5/gamma_diffuse/LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0003.hdf5'
f_4 = h5py.File(file_4, 'r')
file_5 = '/home/jacquemm/hdf5/gamma_diffuse/LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0010.hdf5'
f_5 = h5py.File(file_5, 'r')
file_6 = '/home/jacquemm/hdf5/gamma_diffuse/LaPalma_gamma_diffuse_20deg_0deg_prod3b_training_0011.hdf5'
f_6 = h5py.File(file_6, 'r')
injTable = np.array(f_1['/Cameras/LSTCAM/injTable'])
nbRow = f_1['/Cameras/LSTCAM'].attrs['nbRow']
nbCol = f_1['/Cameras/LSTCAM'].attrs['nbCol']
dataset_1 = LSTCamDataset(hdf5_file=f_1,
                             transform=transforms.Compose([
                                 TelescopeToSquareMatrix(injTable, nbRow, nbCol)
                             ]))
dataset_2 = LSTCamDataset(hdf5_file=f_2,
                             transform=transforms.Compose([
                                 TelescopeToSquareMatrix(injTable, nbRow, nbCol)
                             ]))
dataset_3 = LSTCamDataset(hdf5_file=f_3,
                             transform=transforms.Compose([
                                 TelescopeToSquareMatrix(injTable, nbRow, nbCol)
                             ]))
dataset_4 = LSTCamDataset(hdf5_file=f_4,
                             transform=transforms.Compose([
                                 TelescopeToSquareMatrix(injTable, nbRow, nbCol)
                             ]))
dataset_5 = LSTCamDataset(hdf5_file=f_6,
                             transform=transforms.Compose([
                                 TelescopeToSquareMatrix(injTable, nbRow, nbCol)
                             ]))
dataset_test = LSTCamDataset(hdf5_file=f_6,
                             transform=transforms.Compose([
                                 TelescopeToSquareMatrix(injTable, nbRow, nbCol)
                             ]))


dataset_train = ConcatDataset([dataset_1, dataset_2, dataset_3, dataset_4, dataset_5])


In [15]:
len(dataset_train)

34676

In [6]:
# iterable subset creation
train_loader = torch.utils.data.DataLoader(dataset=dataset_train,
                                           batch_size=len(dataset_train))

test_loader = torch.utils.data.DataLoader(dataset=dataset_test,
                                          batch_size=len(dataset_test))

In [16]:
with h5py.File('/home/jacquemm/hdf5/converted/gamma_diffuse_train', 'w') as f_train:
    for i, samples in enumerate(train_loader):
        f_train.create_dataset('images', data=samples['image'].numpy(), dtype=np.float32)
        f_train.create_dataset('telescopes', data=samples['telescope'].numpy(), dtype=np.float32)
        f_train.create_dataset('labels', data=samples['labels'].numpy(), dtype=np.float32)

In [13]:
with h5py.File('/home/jacquemm/hdf5/converted/gamma_diffuse_test', 'w') as f_test:
    for i, samples in enumerate(test_loader):
        f_test.create_dataset('images', data=samples['image'].numpy(), dtype=np.float32)
        f_test.create_dataset('telescopes', data=samples['telescope'].numpy(), dtype=np.float32)
        f_test.create_dataset('labels', data=samples['labels'].numpy(), dtype=np.float32)

In [ ]:
f_1.close()
f_2.close()
f_3.close()
f_4.close()
f_5.close()
f_6.close()